<a href="https://colab.research.google.com/github/mjgpinheiro/Physics_models/blob/main/Stock_Trading_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import time
# Get the current timestamp
#timestamp = int(time.time())
# Define the notebook name
#notebook_name = f"stock_trading_bot_{timestamp}.ipynb"
# Save the notebook
#!jupyter nbconvert --to notebook --output-dir='./' --output={notebook_name} --execute notebook_name.ipynb

# Import necessary libraries
import tensorflow as tf
import numpy as np
import pandas as pd
import yfinance as yf
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM

# Step 1: Setting Up Your Python Environment
# Install necessary packages
# !pip install tensorflow
# !pip install numpy
# !pip install pandas
# !pip install yfinance
# !pip install scikit-learn

# Step 2: Gathering Data for Your Stock Trading Bot
# Collect historical price data
#symbol = "AAPL"
#symbol = "MSFT"
symbol = "GC"

start_date = "2010-01-01"
end_date = "2023-07-13"
df = yf.download(symbol, start=start_date, end=end_date)

# Preprocess the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df["Close"].values.reshape(-1, 1))

# Split the data into training and testing sets
train_size = int(len(scaled_data) * 0.8)
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size:]

# Define a function to create input sequences and labels
def create_sequences(data, seq_length):
    x = []
    y = []
    for i in range(seq_length, len(data)):
        x.append(data[i-seq_length:i, 0])
        y.append(data[i, 0])
    return np.array(x), np.array(y)


# Create input sequences and labels
seq_length = 60
train_x, train_y = create_sequences(train_data, seq_length)
test_x, test_y = create_sequences(test_data, seq_length)

# Step 3: Building Your Stock Trading Bot
# Define the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(train_x.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer="adam", loss="mean_squared_error")

# Train the model
model.fit(train_x, train_y, epochs=10, batch_size=32)

# Step 4: Evaluating Your Stock Trading Bot
# Evaluate the model on the test set
test_loss = model.evaluate(test_x, test_y)

# Step 5: Using Your Stock Trading Bot
# Make predictions on the test set
predictions = model.predict(test_x)
predictions = scaler.inverse_transform(predictions)

# Convert the predictions and actual values to pandas DataFrame
predictions_df = pd.DataFrame(predictions, index=df.index[train_size+seq_length:])
actual_df = pd.DataFrame(scaler.inverse_transform(test_y.reshape(-1, 1)), index=df.index[train_size+seq_length:])

# Compare the predictions and actual values
comparison_df = pd.concat([actual_df, predictions_df], axis=1)
comparison_df.columns = ["Actual", "Predicted"]
print(comparison_df)

# Save the model for deployment
model.save("stock_trading_bot_model.keras")

# Create input sequences and labels
seq_length = 60
train_x, train_y = create_sequences(train_data, seq_length)
test_x, test_y = create_sequences(test_data, seq_length)

# Step 3: Building Your Stock Trading Bot
# Define the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(train_x.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(units=1))

# Define the input shape for predicting the next day
next_day_sequence = np.expand_dims(test_x[-1], axis=0)

# Make a prediction for the next day
next_day_prediction = model.predict(next_day_sequence)
next_day_price = scaler.inverse_transform(next_day_prediction)[0][0]

# Print the predicted price for the next day
print("Predicted price for the next day:", next_day_price)

# Determine the action based on the predicted price
if next_day_price > df["Close"].iloc[-1]:
    print("Action: Buy")
elif next_day_price < df["Close"].iloc[-1]:
    print("Action: Sell")
else:
    print("Action: Hold")


[*********************100%***********************]  1 of 1 completed
Epoch 1/10
31/31 [==============================] - 13s 62ms/step - loss: 3.2657e-04
Epoch 2/10
31/31 [==============================] - 2s 62ms/step - loss: 7.9950e-05
Epoch 3/10
31/31 [==============================] - 2s 61ms/step - loss: 3.2660e-05
Epoch 4/10
31/31 [==============================] - 2s 73ms/step - loss: 2.4905e-05
Epoch 5/10
31/31 [==============================] - 3s 89ms/step - loss: 2.5122e-05
Epoch 6/10
31/31 [==============================] - 2s 61ms/step - loss: 2.4458e-05
Epoch 7/10
31/31 [==============================] - 2s 61ms/step - loss: 2.4347e-05
Epoch 8/10
31/31 [==============================] - 2s 60ms/step - loss: 2.5029e-05
Epoch 9/10
31/31 [==============================] - 2s 61ms/step - loss: 2.4136e-05
Epoch 10/10
7/7 [==============================] - 1s 17ms/step
            Actual  Predicted
Date                         
2014-06-20    4.46   4.452291
2014-06-23    4.44  